## Importing Libraries

In [1]:
#Import Libraries
import pandas as pd
import numpy as np
import os

## Importing Data Set

In [2]:
#Create Path
path = r'C:\Users\praha\OneDrive\Documents\Python\02-2023 Instachart Basket Analysis'

In [3]:
#import dataframe 'orders_products_merged_2.pkl'
ords_prods_merge = pd.read_pickle(os.path.join(path, '02. Data', 'Prepared Data', 'orders_products_merged_2.pkl'))

In [5]:
#create subset of the first one million entries
df = ords_prods_merge[:1000000]

In [6]:
df.shape

(1000000, 18)

## Find aggregated mean of the 'Order_number' column grouped by 'department_id'

In [7]:
#calculate the mean of the 'order_number' column grouped by the 'department_id' from subset
df.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.577493
2,17.320781
3,16.084944
4,17.530458
5,14.763075
6,16.658449
7,17.031590
8,15.076662


In [4]:
#calculate the mean of the 'order_number' column grouped by the 'department_id' from entire dataframe
ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


Analysis: subset and entire dataframe have slightly different means. Some of subset means are lower than entire dataframe, and other are higher. 

## Creating a Loyalty Flag for Existing Customers using the Transform() and Loc()

In [27]:
#agggregate data with transform()
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [28]:
#add loc() functions
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [29]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [30]:
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [31]:
#count the value of 'loyalty_flag'
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: loyalty_flag, dtype: int64

In [32]:
ords_prods_merge.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,...,_merge,price_label,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,max_prices,max_price,Spending_flag
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,...,both,Mid-range-product,regularly busy,regularly busy,Most Orders,32,Regular customer,20.0,20.0,NaN
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,...,both,Mid-range-product,regularly busy,regularly busy,Average Orders,32,Regular customer,20.0,20.0,NaN
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,...,both,Mid-range-product,Busiest day,Busiest days,Average Orders,5,New customer,18.5,18.5,NaN
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,...,both,Mid-range-product,regularly busy,Slowest days,Most Orders,3,New customer,14.0,14.0,NaN
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,...,both,Mid-range-product,least busy,Slowest days,Average Orders,3,New customer,14.0,14.0,NaN


## Checking the basic statistics of the product prices for each loyalty category

In [33]:
#check the basic statistics of the product prices for each loyalty category
ords_prods_merge.groupby('loyalty_flag').agg({'prices':['count', 'mean', 'min', 'max']})

prices                         
                     count       mean  min      max
loyalty_flag                                       
Loyal customer    10284093  10.386336  1.0  99999.0
New customer       6243990  13.294670  1.0  99999.0
Regular customer  15876776  12.495717  1.0  99999.0

Analysis: The basic statistics performed in this analysis are count, mean, min, and max. As we can see, the highest number of customer owned by regular customer category. All categories have same minimum and maximum prices. The loyal customer category has the lowest mean of prices than others, which means, although the loyal customers buy the products with the lowest prices than other categories, but they are loyal to the brand. 

## Creating a Spending Flag for Existing Customers using the Transform() and Loc()

In [47]:
#agggregate data with transform()
ords_prods_merge['average_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

In [49]:
#add loc() functions
ords_prods_merge.loc[ords_prods_merge['average_price'] < 10, 'Spending_flag'] = 'Low spender'

In [50]:
ords_prods_merge.loc[ords_prods_merge['average_price'] >= 10, 'Spending_flag'] = 'High spender'

In [51]:
ords_prods_merge.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,...,price_label,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,max_prices,max_price,Spending_flag,average_price
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,...,Mid-range-product,regularly busy,regularly busy,Most Orders,32,Regular customer,20.0,6.935811,Low spender,6.935811
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,...,Mid-range-product,regularly busy,regularly busy,Average Orders,32,Regular customer,20.0,6.935811,Low spender,6.935811
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,...,Mid-range-product,Busiest day,Busiest days,Average Orders,5,New customer,18.5,7.930208,Low spender,7.930208
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,...,Mid-range-product,regularly busy,Slowest days,Most Orders,3,New customer,14.0,4.972414,Low spender,4.972414
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,...,Mid-range-product,least busy,Slowest days,Average Orders,3,New customer,14.0,4.972414,Low spender,4.972414


In [52]:
#check the basic statistics of the product prices for each spending category
ords_prods_merge.groupby('Spending_flag').agg({'prices':['count', 'mean', 'min', 'max']})

prices                          
                  count        mean  min      max
Spending_flag                                    
High spender     634245  222.343698  1.0  99999.0
Low spender    31770614    7.780685  1.0     25.0

## Creating a Frequency Flag for Existing Customers using the Transform() and Loc()

In [53]:
ords_prods_merge.columns

Index(['product_id', 'product_name', 'aisle_id', 'department_id', 'prices',
       'order_id', 'user_id', 'order_number', 'orders_day_of_week',
       'order_hour_of_day', 'days_since_prior_order', 'add_to_cart_order',
       'reordered', '_merge', 'price_label', 'busiest_day', 'busiest_days',
       'busiest_period_of_day', 'max_order', 'loyalty_flag', 'max_prices',
       'max_price', 'Spending_flag', 'average_price'],
      dtype='object')

In [54]:
#agggregate data with transform()
ords_prods_merge['regularity_of_users_ordering_behavior'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [55]:
#add loc() functions
ords_prods_merge.loc[ords_prods_merge['regularity_of_users_ordering_behavior'] > 20, 'frequency_flag'] = 'Non-frequent customer'

In [56]:
#add loc() functions
ords_prods_merge.loc[(ords_prods_merge['regularity_of_users_ordering_behavior'] > 10) & (ords_prods_merge['regularity_of_users_ordering_behavior'] <= 20), 'frequency_flag'] = 'Regular customer'

In [57]:
ords_prods_merge.loc[ords_prods_merge['regularity_of_users_ordering_behavior'] <= 10, 'frequency_flag'] = 'Frequent customer'

In [58]:
ords_prods_merge.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,...,busiest_days,busiest_period_of_day,max_order,loyalty_flag,max_prices,max_price,Spending_flag,average_price,regularity_of_users_ordering_behavior,frequency_flag
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,...,regularly busy,Most Orders,32,Regular customer,20.0,6.935811,Low spender,6.935811,8.0,Frequent customer
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,...,regularly busy,Average Orders,32,Regular customer,20.0,6.935811,Low spender,6.935811,8.0,Frequent customer
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,...,Busiest days,Average Orders,5,New customer,18.5,7.930208,Low spender,7.930208,7.0,Frequent customer
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,...,Slowest days,Most Orders,3,New customer,14.0,4.972414,Low spender,4.972414,9.0,Frequent customer
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,...,Slowest days,Average Orders,3,New customer,14.0,4.972414,Low spender,4.972414,9.0,Frequent customer


In [60]:
#check the basic statistics of the product prices for each frequency category
ords_prods_merge.groupby('frequency_flag').agg({'days_since_prior_order':['count', 'mean', 'min', 'max']})

days_since_prior_order                      
                                       count       mean  min   max
frequency_flag                                                    
Frequent customer                   22790523   8.136934  0.0  30.0
Non-frequent customer                2686728  21.718032  0.0  30.0
Regular customer                     6927608  15.519100  0.0  30.0

In [61]:
#export the data to "ords_prods_merge.pkl"
ords_prods_merge.to_pickle(os.path.join(path, '02. Data', 'Prepared Data', 'orders_products_merged_3.pkl'))